In [1]:
from time import time
from numpy import log, sqrt, exp
import numpy as np
import pandas as pd
from scipy.stats import norm

np.set_printoptions(precision = 4)

In [2]:
def gen_stock_price(S0, rands = None, rows = None, cols = None, time = None):
    '''Generate the value of stock [based on given random variable]'''
    if rands is None:    # if no random variable, generate random standard normal variable based on given rows and cols
        if rows is None or cols is None: raise ValueError("Must take rands or rows and cols as input")
        randoms = np.random.randn(rows, cols)
    else:
        rows = len(randoms)    # otherwises, we use the passed-in data and we get its number of row
        randoms = rands.copy()
    if time is None:    # if no time interval, default to be 1/365
        time = np.full(shape = rows, fill_value = 1/365)
    
    randoms[0] = S0 
    for i in range(1,rows):
        randoms[i] = randoms[i-1] * exp((r-q-v**2/2)*time[i] + v*sqrt(time[i]) * randoms[i])
    
    return randoms

def get_date_interval(dates):
    '''get the interval of dates, used as dt to derive stock price'''
    l = len(dates)
    interval = list((dates[i]-dates[i-1]).days for i in range(l))
    return np.array(interval)
def get_act_date(dates):
    '''get the time range from start date to each date after, used for discounting'''
    l = len(dates)
    act = list((dates[i]-dates[0]).days for i in range(l))
    return np.array(act)

def eval_option(path, K, rebate = 0, DI = None, UO = None):
    '''
    return the rebate value or final value and the computed date of such value. e.g. value, date number
    
    path: a path of stock value pd.Series
    
    DI,UO: optional pd.Series
    '''
    l = len(path)
    path_v = path.values
        
    if UO is not None:
        UO_v = UO.values
        loc = np.argmax(path_v>UO_v) # argmax return the location of first largest, useful here
        if loc>0: return rebate, loc
    
    if DI is not None:
        DI_v = DI.values
        DI_flag = np.any(path_v<DI_v) # DI for once
    else:
        DI_flag = 1
    
    value = max(path_v[-1]-K,0) * DI_flag
    return value, l-1

def cal_mc_result(S0, K, r, q, v, mcpath, dates, rands=None, event=None, rebate = 0):
    '''
    main monte carlo program, can accept predefined random variables to realize antithetic MC
    
    can consider UO or DI event,
    
    return an array of discounted value, length = mcpath
    '''
    ###### generate stock price
    dates_num = len(dates)
    date_interval = get_date_interval(dates) / 365
    if rands is None:
        stock_data = gen_stock_price(S0, rows = dates_num, cols = mcpath, time = date_interval)
    else:
        stock_data = gen_stock_price(S0, rands = rands, time = date_interval)
    stock = pd.DataFrame(stock_data, columns = list(range(mcpath)), index = dates)
    
    ###### 
    discount = exp(get_act_date(dates) / 365 * r)
    results = np.zeros(mcpath)

    UO = event['UO'] if 'UO' in event.columns else None
    DI = event['DI'] if 'DI' in event.columns else None
        
    for path in range(mcpath):
        val, date = eval_option(stock[path], K, rebate=rebate, UO=UO, DI=DI)
        results[path] = val * discount[date]

    return results

In [3]:
def barrier_option(S, K, H, r, q, sig, t, otype, R=0):
    """return the analytical solution of american barrier"""
    def help(eta, phi, coe):
        f = [0]*6
        f[0] = phi*S*d**(-t)*norm.cdf(phi*x)  - phi*K*r**(-t)*norm.cdf(phi*x -  phi*sig*sqrt(t))
        f[1] = phi*S*d**(-t)*norm.cdf(phi*x1) - phi*K*r**(-t)*norm.cdf(phi*x1 - phi*sig*sqrt(t))

        f[2] = phi*S*d**(-t)*(H/S)**(2*_lambda)*norm.cdf(eta*y) - phi*K*r**(-t)*(H/S)**(2*_lambda-2)*norm.cdf(eta*y - eta*sig*sqrt(t))
        f[3] = phi*S*d**(-t)*(H/S)**(2*_lambda)*norm.cdf(eta*y1) - phi*K*r**(-t)*(H/S)**(2*_lambda-2)*norm.cdf(eta*y1 - eta*sig*sqrt(t))

        f[4] = R*r**(-t)*(norm.cdf(eta*x1 - eta*sig*sqrt(t)) - (H/S)**(2*_lambda-2)*norm.cdf(eta*y1 - eta*sig*sqrt(t)))
        f[5] = R*((H/S)**(a+b)*norm.cdf(eta*z) + (H/S)**(a-b)*norm.cdf(eta*z - 2*eta*b*sig*sqrt(t)))
        
        return sum(f[i]*coe[i] for i in range(6))
    
    # t can be an array of day or just the fraction of year
    if hasattr(t, "__len__"):
        t = len(t)/365
    
    r, d = r+1, q+1

    miu = log(r/d) - sig**2/2
    _lambda = (1+miu/sig**2)
    a = miu/sig**2
    b = sqrt(miu**2 +2*log(r)*sig**2)/sig**2

    x  = log(S/K)     /sig/sqrt(t) + _lambda*sig*sqrt(t)
    x1 = log(S/H)     /sig/sqrt(t) + _lambda*sig*sqrt(t)
    y  = log(H**2/S/K)/sig/sqrt(t) + _lambda*sig*sqrt(t)
    y1 = log(H/S)     /sig/sqrt(t) + _lambda*sig*sqrt(t)
    z  = log(H/S)     /sig/sqrt(t) + b      *sig*sqrt(t)

    if otype == "CUO" and K<=H: return help(-1, 1, ( 1,-1, 1,-1, 0, 1))
    if otype == "CUO" and K> H: return help(-1, 1, ( 0, 0, 0, 0, 0, 1))
    if otype == "CUI" and K<=H: return help(-1, 1, ( 0, 1,-1, 1, 1, 0))
    if otype == "CUI" and K> H: return help(-1, 1, ( 1, 0, 0, 0, 1, 0))
    
    if otype == "CDO" and K<=H: return help( 1, 1, ( 0, 1, 0,-1, 0, 1))
    if otype == "CDO" and K> H: return help( 1, 1, ( 1, 0,-1, 0, 0, 1))
    if otype == "CDI" and K<=H: return help( 1, 1, ( 1,-1, 0, 1, 1, 0))
    if otype == "CDI" and K> H: return help( 1, 1, ( 0, 0, 1, 0, 1, 0))
    
    if otype == "PUO" and K<=H: return help(-1,-1, ( 1, 0,-1, 0, 0, 1))
    if otype == "PUO" and K> H: return help(-1,-1, ( 0, 1, 0,-1, 0, 1))
    if otype == "PUI" and K<=H: return help(-1,-1, ( 0, 0, 1, 0, 1, 0))
    if otype == "PUI" and K> H: return help(-1,-1, ( 1,-1, 0, 1, 1, 0))
    
    if otype == "PDO" and K<=H: return help( 1,-1, ( 0, 0, 0, 0, 0, 1))
    if otype == "PDO" and K> H: return help( 1,-1, ( 1,-1, 1,-1, 0, 1))
    if otype == "PDI" and K<=H: return help( 1,-1, ( 1, 0, 0, 0, 1, 0))
    if otype == "PDI" and K> H: return help( 1,-1, ( 0, 1,-1, 1, 1, 0))
    
    raise KeyError("otype has wrong input")

## Barrier Option Pricing
spot: 100 <br>
strike: 105 <br>
maturity: 6 months <br>
volatility: 30% <br>
no dividend <br>
riskfree rate: 2%

In [4]:
# Some preset parameters
S0, K, r, q, v = 100., 105., .02, 0., .3
mcpath = 10000 # number of path for one Monte Carlo
dates = pd.bdate_range(start = pd.datetime(2020,6,1), end = pd.datetime(2020,11,30))
dates_num = len(dates)

## Case 1
Constant Up and Out barrier at 110 effective in the last 3 months 

In [5]:
# simple MC
time1 = time()
rebate = 0

event = pd.DataFrame(None, columns = ['UO'], index = dates)
event['UO'] = 9999
event.loc[event.index>=pd.datetime(2020,9,1),'UO'] = 110
rands = np.random.randn(dates_num, mcpath)

result = cal_mc_result(S0, K, r, q, v, mcpath, dates, rands, event, rebate)

time2 = time()
print("It takes {:.4f} sec to do the MC with control variates with {:d} paths\n".format(time2-time1, mcpath))
print("The result is, {:.6f}, with a variance of {:.6f}\n".format(np.mean(result), np.var(result)))

It takes 0.4743 sec to do the MC with control variates with 10000 paths

The result is, 0.035912, with a variance of 0.109633



In [6]:
# antithetic
time1 = time()
rebate = 0

event = pd.DataFrame(None, columns = ['UO'], index = dates)
event['UO'] = 9999
event.loc[event.index>=pd.datetime(2020,9,1),'UO'] = 110
rands = np.random.randn(dates_num, mcpath)

result     = cal_mc_result(S0, K, r, q, v, mcpath, dates,  rands, event, rebate)
result_anti= cal_mc_result(S0, K, r, q, v, mcpath, dates, -rands, event, rebate)
result = (result + result_anti)/2

time2 = time()
print("It takes {:.4f} sec to do the MC with control variates with {:d} paths\n".format(time2-time1, mcpath))
print("The result is, {:.6f}, with a variance of {:.6f}\n".format(np.mean(result), np.var(result)))

It takes 0.8703 sec to do the MC with control variates with 10000 paths

The result is, 0.033265, with a variance of 0.045329



In [7]:
# control variate
time1 = time()

event = pd.DataFrame(None, columns = ['UO'], index = dates)
event_c = event.copy()
event['UO'] = 9999
event.loc[event.index>=pd.datetime(2020,9,1),'UO'] = 110
event_c['UO'] = 110 # how to select

rands = np.random.randn(dates_num, mcpath)
rebate = 0

result   = cal_mc_result(S0, K, r, q, v, mcpath, dates,  rands, event  , rebate)
result_c = cal_mc_result(S0, K, r, q, v, mcpath, dates,  rands, event_c, rebate)

beta = np.cov(result,result_c)[0,1] / np.var(result_c)

H_c = 110*exp(0.5*v*sqrt(1/762))
otype = "CUO"
v_c = barrier_option(S0, K, H_c, r, q, v, dates, otype, rebate)
print("cor = {:.6f}, beta = {:.6f}, analytical value = {:6f}, control mean = {:6f}\n".format(
    np.corrcoef(result,result_c)[0,1], beta, v_c, np.mean(result_c)))

result  = result + beta * (v_c - result_c)

time2 = time()
print("It takes {:.4f} sec to do the MC with control variates with {:d} paths\n".format(time2-time1, mcpath))
print("The result is, {:.6f}, with a variance of {:.6f}\n".format(np.mean(result), np.var(result)))

cor = 0.854781, beta = 0.996688, analytical value = 0.028254, control mean = 0.028633

It takes 1.2908 sec to do the MC with control variates with 10000 paths

The result is, 0.038572, with a variance of 0.031702



## Case 2
Step-up barrier: 108 for the first 3 months, 110 in the last 3 months

In [8]:
time1 = time()

event = pd.DataFrame(None, columns = ['UO'], index = dates)
event['UO'] = 9999
event.loc[event.index< pd.datetime(2020,9,1),'UO'] = 108
event.loc[event.index>=pd.datetime(2020,9,1),'UO'] = 110

rands = np.random.randn(dates_num, mcpath)
rebate = 0

result = cal_mc_result(S0, K, r, q, v, mcpath, dates,  rands, event, rebate)

time2 = time()
print("It takes {:.4f} sec to do the MC with control variates with {:d} paths\n".format(time2-time1, mcpath))
print("The result is, {:.6f}, with a variance of {:.6f}\n".format(np.mean(result), np.var(result)))

It takes 0.5087 sec to do the MC with control variates with 10000 paths

The result is, 0.022923, with a variance of 0.068717



## Case 3
Linearly increasing barrier: From 105 to 115 during the 6 months

In [9]:
time1 = time()

event = pd.DataFrame(None, columns = ['UO'], index = dates)
event['UO'] = np.linspace(105,115,dates_num)

rands = np.random.randn(dates_num, mcpath)
rebate = 0

result = cal_mc_result(S0, K, r, q, v, mcpath, dates,  rands, event, rebate)

time2 = time()
print("It takes {:.4f} sec to do the MC with control variates with {:d} paths\n".format(time2-time1, mcpath))
print("The result is, {:.6f}, with a variance of {:.6f}\n".format(np.mean(result), np.var(result)))

It takes 0.4399 sec to do the MC with control variates with 10000 paths

The result is, 0.092541, with a variance of 0.487719



In [10]:
# control variate
time1 = time()
rebate = 0

event = pd.DataFrame(None, columns = ['UO'], index = dates)
event_c = event.copy()

event['UO'] = np.linspace(105,115,dates_num)
event_c['UO'] = 110 # how to calculate????
rands = np.random.randn(dates_num, mcpath)

result   = cal_mc_result(S0, K, r, q, v, mcpath, dates,  rands, event  , rebate)
result_c = cal_mc_result(S0, K, r, q, v, mcpath, dates,  rands, event_c, rebate)

beta = np.cov(result,result_c)[0,1] / np.var(result_c)
H_c = 110*exp(0.5*v*sqrt(1/732))
otype = "CUO"
v_c = barrier_option(S0, K, H_c, r, q, v, dates, otype, rebate)
print("cor = {:.6f}, beta = {:.6f}, analytical value = {:6f}, control mean = {:6f}\n".format(
    np.corrcoef(result, result_c)[0,1], beta, v_c, np.mean(result_c)))

result  = result + beta * (v_c - result_c)

time2 = time()
print("It takes {:.4f} sec to do the MC with control variates with {:d} paths\n".format(time2-time1, mcpath))
print("The result is, {:.6f}, with a variance of {:.6f}\n".format(np.mean(result), np.var(result)))

cor = 0.288591, beta = 0.687106, analytical value = 0.028454, control mean = 0.025774

It takes 0.8567 sec to do the MC with control variates with 10000 paths

The result is, 0.078477, with a variance of 0.378837



## Case 4
Constantly discretely sampled barrier: Barrier at 110 sampled monthly, weekly and daily (3 separate cases) 

In [11]:
# monthly
time1 = time()

event = pd.DataFrame(None, columns = ['UO'], index = dates)
month = event.resample('BM').last()
event.loc[month.index, "UO"] = 110

rands = np.random.randn(dates_num, mcpath)
rebate = 0

result = cal_mc_result(S0, K, r, q, v, mcpath, dates,  rands, event, rebate)

time2 = time()
print("It takes {:.4f} sec to do the MC with control variates with {:d} paths\n".format(time2-time1, mcpath))
print("The result is, {:.6f}, with a variance of {:.6f}\n".format(np.mean(result), np.var(result)))

D:\Apps\Program\Anaconda\lib\site-packages\ipykernel_launcher.py:42: RuntimeWarning: invalid value encountered in greater


It takes 0.8802 sec to do the MC with control variates with 10000 paths

The result is, 0.075731, with a variance of 0.234922



In [12]:
# monthly control variate
time1 = time()

event = pd.DataFrame(None, columns = ['UO'], index = dates)
event_c = event.copy()
month = event.resample('BM').last()
event.loc[month.index, "UO"] = 110
event_c['UO'] = 110*exp(0.5*v*sqrt(30/365)) # how to calculate????

rands = np.random.randn(dates_num, mcpath)
rebate = 0

result   = cal_mc_result(S0, K, r, q, v, mcpath, dates,  rands, event  , rebate)
result_c = cal_mc_result(S0, K, r, q, v, mcpath, dates,  rands, event_c, rebate)

beta = np.cov(result,result_c)[0,1] / np.var(result_c)
H_c = 110*exp(0.5*v*sqrt(30/365))
otype = "CUO"
v_c = barrier_option(S0, K, H_c, r, q, v, dates, otype, rebate)
print("cor = {:.6f}, beta = {:.6f}, analytical value = {:6f}, control mean = {:6f}".format(
    np.corrcoef(result, result_c)[0,1], beta, v_c, np.mean(result_c)))

result  = result + beta * (v_c - result_c)

time2 = time()
print("It takes {:.4f} sec to do the MC with control variates with {:d} paths\n".format(time2-time1, mcpath))
print("The result is, {:.6f}, with a variance of {:.6f}\n".format(np.mean(result), np.var(result)))

D:\Apps\Program\Anaconda\lib\site-packages\ipykernel_launcher.py:42: RuntimeWarning: invalid value encountered in greater


cor = 0.384965, beta = 0.192078, analytical value = 0.170275, control mean = 0.175653
It takes 0.9874 sec to do the MC with control variates with 10000 paths

The result is, 0.073676, with a variance of 0.208276



In [13]:
# weekly
time1 = time()

event = pd.DataFrame(None, columns = ['UO'], index = dates)
month = event[event.index.dayofweek == 4] # take 4th day of the week, 0 is Monday
event.loc[month.index, "UO"] = 110

rands = np.random.randn(dates_num, mcpath)
rebate = 0

result = cal_mc_result(S0, K, r, q, v, mcpath, dates,  rands, event, rebate)

time2 = time()
print("It takes {:.4f} sec to do the MC with control variates with {:d} paths\n".format(time2-time1, mcpath))
print("The result is, {:.6f}, with a variance of {:.6f}\n".format(np.mean(result), np.var(result)))

D:\Apps\Program\Anaconda\lib\site-packages\ipykernel_launcher.py:42: RuntimeWarning: invalid value encountered in greater


It takes 0.6991 sec to do the MC with control variates with 10000 paths

The result is, 0.062253, with a variance of 0.277482



In [14]:
# daily
time1 = time()

event = pd.DataFrame(None, columns = ['UO'], index = dates)
event["UO"] = 110

rands = np.random.randn(dates_num, mcpath)
rebate = 0

result = cal_mc_result(S0, K, r, q, v, mcpath, dates,  rands, event, rebate)

time2 = time()
print("It takes {:.4f} sec to do the MC with control variates with {:d} paths\n".format(time2-time1, mcpath))
print("The result is, {:.6f}, with a variance of {:.6f}\n".format(np.mean(result), np.var(result)))

It takes 0.4823 sec to do the MC with control variates with 10000 paths

The result is, 0.023404, with a variance of 0.067236

